In [1]:
import pandas as pd
import numpy as np
import requests
import os
import sys

pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

import psycopg2
from psycopg2 import sql
# Ignorar warings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

sys.path.append(os.path.abspath("src"))   
import src_funciones as f

In [2]:
datos_artista=pd.read_csv("datos/detalles_artistas.csv")
datos_pelis=pd.read_csv("datos/detalles_peliculas.csv")
pelis=pd.read_csv("datos/peliculas.csv")
peli_artista=pd.read_csv("datos/pelicula_artista.csv")

In [3]:
datos_artista.head(3)

,actr,año_nac,conocido_por,que_hace,premios
0,A. Gunaseelan,9999,'Mandela',"Casting Department, Actress, Casting Director",1 nomination
1,A. Michelle Harleston,9999,"'Murder by Numbers', 'The Mark of a Killer', '...",Actor,no tiene premios
2,A.C. Peterson,9999,"'Narc', 'Shanghai Noon', 'Shooter', 'The Last ...",Actor,no tiene premios


In [4]:
datos_artista.duplicated().sum()

np.int64(0)

In [5]:
datos_artista.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9907 entries, 0 to 9906
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   actr          9907 non-null   object
 1   año_nac       9907 non-null   int64 
 2   conocido_por  9907 non-null   object
 3   que_hace      9907 non-null   object
 4   premios       9907 non-null   object
dtypes: int64(1), object(4)
memory usage: 387.1+ KB


In [6]:
datos_artista["año_nac"]=datos_artista["año_nac"].astype(str)
datos_artista["año_nac"]=datos_artista["año_nac"].str.replace("9999", "desconocido")

In [7]:
datos_artista["año_nac"].unique()

array(['desconocido', '1957', '1945', '1975', '1969', '1966', '1991',
       '1980', '1952', '1937', '1958', '1990', '1994', '2009', '2000',
       '1968', '1979', '1970', '1982', '1934', '1986', '2006', '1976',
       '1955', '1933', '1972', '1999', '1989', '1977', '1981', '1993',
       '1921', '1984', '1936', '1963', '1997', '1960', '1978', '1938',
       '1973', '1971', '1964', '1962', '1995', '1983', '1988', '1914',
       '1974', '1985', '1967', '1948', '1946', '1956', '1935', '1943',
       '1927', '1920', '1961', '2004', '1928', '1947', '1913', '1987',
       '1965', '1950', '1953', '1954', '1931', '1942', '1932', '1992',
       '1941', '2001', '1951', '1891', '1996', '1924', '1959', '1940',
       '1949', '1944', '1893', '1919', '1915', '1926', '2005', '1929',
       '1910', '1879', '1939', '1923', '1907', '1930', '1925', '2007',
       '1998', '1922', '1916', '2002', '1908', '1911', '1918', '1917',
       '1904', '1897', '2003', '1898', '2011', '1899', '2008', '1876',
       

In [8]:
len(datos_artista["premios"].unique())

872

Son únicos (pero o bien faltan aquí 2 o bien sobran 2 en el df llamado peli_artista)

In [9]:
len(datos_artista["actr"].unique())

9907

In [10]:
datos_artista.head()

,actr,año_nac,conocido_por,que_hace,premios
0,A. Gunaseelan,desconocido,'Mandela',"Casting Department, Actress, Casting Director",1 nomination
1,A. Michelle Harleston,desconocido,"'Murder by Numbers', 'The Mark of a Killer', '...",Actor,no tiene premios
2,A.C. Peterson,desconocido,"'Narc', 'Shanghai Noon', 'Shooter', 'The Last ...",Actor,no tiene premios
3,A.V.S. Subramanyam,1957,"'Roommates', 'Uncle', 'Ori Nee Prema Bangaramk...","Actor, Director, Writer",no tiene premios
4,Øyvind Øvrebø,desconocido,"'Tingen, Edderkoppen', 'Demenskoret', 'Kristia...","Actress, Additional Crew",no tiene premios


In [11]:
datos_pelis.head(3)

,titulo,cali_imdb,direc_imbd,guion_imbd,argum_imbd,durac_imbd,id_pelicula
0,Las Travesuras de Super Chido,7.7,Alejandro Todd,Óscar FentanesJuan Garrido,Desconocido,1h 35min,tt0320656
1,Velai Kidaichiruchu,5.2,P. Vasu,SathyarajGautamiAnandraj,"Baasha, a gangster, controls a whole village w...",Desconocido,tt0318803
2,Sor Batalla,6.6,Jesús Fragoso Montoya,Fernando Galiana,As punishment for participating in a contest T...,1h 34min,tt0320503


In [ ]:
datos_pelis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1453 entries, 0 to 1452
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   titulo       1453 non-null   object 
 1   cali_imdb    961 non-null    float64
 2   direc_imbd   1453 non-null   object 
 3   guion_imbd   1453 non-null   object 
 4   argum_imbd   1453 non-null   object 
 5   durac_imbd   1453 non-null   object 
 6   id_pelicula  1453 non-null   object 
dtypes: float64(1), object(6)
memory usage: 79.6+ KB


Estos nulos habría que imputarlos (aunque al observarlos me he dado cuenta que gran parte de esas filas no aportan información)

In [13]:
datos_pelis["cali_imdb"]

0       7.7
1       5.2
2       6.6
3       5.3
4       NaN
       ... 
1448    6.2
1449    NaN
1450    NaN
1451    NaN
1452    NaN
Name: cali_imdb, Length: 1453, dtype: float64

53 filas son todo nan


In [14]:
len(datos_pelis[(datos_pelis["cali_imdb"].isna()) & (datos_pelis["direc_imbd"] == "Desconocido") & (datos_pelis["guion_imbd"] == "Desconocido") & (datos_pelis["argum_imbd"] == "Desconocido") & (datos_pelis["durac_imbd"] == "Desconocido")])

53

189 son todo nan salvo duración

In [15]:
len(datos_pelis[(datos_pelis["cali_imdb"].isna()) & (datos_pelis["direc_imbd"] == "Desconocido") & (datos_pelis["guion_imbd"] == "Desconocido") & (datos_pelis["argum_imbd"] == "Desconocido")])

189

192 son todo nan salvo duración y argumento/guion

In [16]:
len(datos_pelis[(datos_pelis["cali_imdb"].isna()) & (datos_pelis["direc_imbd"] == "Desconocido") & (datos_pelis["argum_imbd"] == "Desconocido")])

192

In [17]:
len(datos_pelis[(datos_pelis["cali_imdb"].isna()) & (datos_pelis["direc_imbd"] == "Desconocido") & (datos_pelis["guion_imbd"] == "Desconocido") ])

192

tengo 854 filas con toda la info completa

In [18]:
datos_pelis[(datos_pelis["cali_imdb"]!=np.nan) & (datos_pelis["direc_imbd"] != "Desconocido") & (datos_pelis["guion_imbd"] != "Desconocido")].shape

(854, 7)

In [19]:
len(datos_pelis["durac_imbd"].unique())

231

In [20]:
datos_pelis["cali_imdb"].unique()

array([7.7, 5.2, 6.6, 5.3, nan, 5.9, 5. , 6.3, 6.7, 4.2, 5.1, 4.4, 3.7,
       6.4, 4.9, 5.8, 6. , 6.8, 4.1, 4. , 5.4, 8.4, 6.2, 7.4, 5.6, 3.2,
       2.8, 7.1, 8.3, 2.9, 7.8, 7.2, 7.3, 7.5, 6.1, 5.7, 4.7, 4.3, 2.4,
       4.5, 7.9, 6.5, 8.7, 7. , 4.6, 5.5, 6.9, 9. , 7.6, 8.1, 4.8, 3.3,
       3. , 8. , 2.6, 8.2, 3.8, 3.5, 3.6, 3.4, 3.9, 2.3, 2.7, 8.5, 1.9,
       2.5, 9.8, 8.9, 9.1, 8.6, 1.3, 9.5, 9.3, 3.1, 2. , 8.8, 9.6, 1.7,
       2.1, 9.2, 9.4, 2.2])

In [21]:
datos_pelis[datos_pelis["cali_imdb"].isna()]

,titulo,cali_imdb,direc_imbd,guion_imbd,argum_imbd,durac_imbd,id_pelicula
4,Suo ming fei dao,NaN,Desconocido,Desconocido,Desconocido,1h 24min,tt0348147
5,Contreras Gang,NaN,Desconocido,Desconocido,Desconocido,1h 51min,tt0349237
7,Jiang hu ba mian feng,NaN,Mei Hu,Feng Ni,The heir to the throne flees to South China an...,1h 27min,tt0351141
9,Alyas Lakay,NaN,Nilo Saez,Jose N. Carreon,Desconocido,1h 45min,tt0344816
13,Que me entierren con la banda,NaN,Alfredo B. Crevenna,Manolo Cárdenas,Desconocido,1h 33min,tt0285962
...,...,...,...,...,...,...,...
1447,Sight,NaN,Hariom Mehta,Hariom Mehta,"A criminal starts killing people in the city, ...",11min,tt27236215
1449,Marne Ke Dar Se Mere Dil,NaN,Priyasha Bhardwaj,Desconocido,"One fateful night , Samadhan reluctantly goes ...",Desconocido,tt27250288
1450,Red Wine,NaN,Desconocido,Desconocido,Desconocido,Desconocido,tt27250168
1451,Magna Lacerta,NaN,Ruby Fuller,Ruby FullerKevin Lucero LessJacob Schindler,Two students produce a documentary about the d...,Desconocido,tt27262238


In [22]:
datos_pelis.duplicated().sum()

np.int64(0)

Si que es única

In [23]:
len(datos_pelis["id_pelicula"].unique())

1453

In [24]:
len(datos_pelis["titulo"].unique())

1444

In [25]:
datos_pelis.head()

,titulo,cali_imdb,direc_imbd,guion_imbd,argum_imbd,durac_imbd,id_pelicula
0,Las Travesuras de Super Chido,7.7,Alejandro Todd,Óscar FentanesJuan Garrido,Desconocido,1h 35min,tt0320656
1,Velai Kidaichiruchu,5.2,P. Vasu,SathyarajGautamiAnandraj,"Baasha, a gangster, controls a whole village w...",Desconocido,tt0318803
2,Sor Batalla,6.6,Jesús Fragoso Montoya,Fernando Galiana,As punishment for participating in a contest T...,1h 34min,tt0320503
3,Azaad Desh Ke Gulam,5.3,S.A. Chandrashekhar,RekhaRishi KapoorJackie Shroff,"Bharti, a law student from a rich family, live...",2h 33min,tt0330019
4,Suo ming fei dao,NaN,Desconocido,Desconocido,Desconocido,1h 24min,tt0348147


In [26]:
pelis.head(3)

,Tipo,Titulo,Año,Mes,Id,generos
0,Movie,Las Travesuras de Super Chido,1990,10,tt0320656,Action
1,Movie,Velai Kidaichiruchu,1990,8,tt0318803,Action
2,Movie,Sor Batalla,1990,8,tt0320503,Action


In [27]:
pelis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1453 entries, 0 to 1452
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Tipo     1453 non-null   object
 1   Titulo   1453 non-null   object
 2   Año      1453 non-null   int64 
 3   Mes      1453 non-null   int64 
 4   Id       1453 non-null   object
 5   generos  1453 non-null   object
dtypes: int64(2), object(4)
memory usage: 68.2+ KB


In [28]:
pelis.duplicated().sum()

np.int64(0)

También es única

In [29]:
len(pelis["Id"].unique())

1453

In [30]:
len(pelis["Titulo"].unique())

1444

In [31]:
pelis.head()

,Tipo,Titulo,Año,Mes,Id,generos
0,Movie,Las Travesuras de Super Chido,1990,10,tt0320656,Action
1,Movie,Velai Kidaichiruchu,1990,8,tt0318803,Action
2,Movie,Sor Batalla,1990,8,tt0320503,Action
3,Movie,Azaad Desh Ke Gulam,1990,4,tt0330019,Action
4,Movie,Suo ming fei dao,1991,11,tt0348147,Action


In [32]:
peli_artista.head(3)

,Movie ID,Actriz_Actor
0,tt0110660,Tony Ka Fai Leung
1,tt0110363,Luke Edwards
2,tt0110397,Jennifer Barker


In [33]:
peli_artista.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14529 entries, 0 to 14528
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Movie ID      14529 non-null  object
 1   Actriz_Actor  14529 non-null  object
dtypes: object(2)
memory usage: 227.1+ KB


In [34]:
peli_artista.duplicated().sum()

np.int64(3298)

3298 duplicados de los cuales 3284 tiene como Actriz_Actor sin datos con lo que dado que solo hay una diferencia de 14 duplicados, me los cargo directamente

In [ ]:
peli_artista[(peli_artista.duplicated()) & (peli_artista["Actriz_Actor"]== "Sin datos") ]

,Movie ID,Actriz_Actor
1607,tt12141548,Sin datos
1613,tt13820914,Sin datos
1622,tt12572368,Sin datos
1647,tt2043811,Sin datos
1655,tt2082492,Sin datos
...,...,...
14523,tt27236215,Sin datos
14524,tt27251096,Sin datos
14525,tt27250288,Sin datos
14526,tt27250168,Sin datos


In [36]:
peli_artista.drop_duplicates(inplace=True)

In [37]:
len(peli_artista["Movie ID"].unique())

1452

In [38]:
len(peli_artista["Actriz_Actor"].unique())

9909

In [39]:
peli_artista.duplicated().sum()

np.int64(0)

In [40]:
peli_artista.shape

(11231, 2)

Ahora cada peli tiene un artista

In [41]:
peli_artista.head()

,Movie ID,Actriz_Actor
0,tt0110660,Tony Ka Fai Leung
1,tt0110363,Luke Edwards
2,tt0110397,Jennifer Barker
3,tt0111045,Miklós Déri
4,tt0111653,Joe Bays


TRAS HABER CONOCIDO LOS DATOS Y HABER SIDO CONSCIENTES DE FUTURAS LIMPIEZAS PASAMOS A LA CREACIÓN DE LAS TABLAS QUE QUEREMOS PARA NUESTRA BASE DE DATOS

EMPIEZO CREANDO LAS TABLAS SIN FK

ANIO

In [42]:

#crear_id(df_anti,nombre_col_antigua,nombre_col_nueva,nombre_col_id_nuevo):


df_anio=f.crear_id(pelis,"Año","anios","id_anio")

In [43]:
df_anio.head()

,id_anio,anios
0,1,1990
1,2,1991
2,3,1992
3,4,1993
4,5,1994


GENERO

In [44]:
df_genero=f.crear_id(pelis,"generos","generos","id_genero")

In [45]:
df_genero.head()

,id_genero,generos
0,1,Action
1,2,Drama
2,3,Comedy
3,4,Fantasy
4,5,Horror


AÑO NACIMIENTO

In [46]:
df_nacimiento=f.crear_id(datos_artista,"año_nac","anio_nac","id_anio_nac")

In [47]:
df_nacimiento.head()

,id_anio_nac,anio_nac
0,1,desconocido
1,2,1957
2,3,1945
3,4,1975
4,5,1969


PREMIOS

In [48]:
df_premios=f.crear_id(datos_artista,"premios","premios","id_premio")

In [49]:
df_premios.head()

,id_premio,premios
0,1,1 nomination
1,2,no tiene premios
2,3,2 nominations
3,4,1 win & 1 nomination
4,5,2 wins & 1 nomination


CALIFICACIONES

In [50]:
df_calificaciones=f.crear_id(datos_pelis,"cali_imdb","calificaciones","id_calificacion")

In [51]:
df_calificaciones

,id_calificacion,calificaciones
0,1,7.7
1,2,5.2
2,3,6.6
3,4,5.3
4,5,NaN
...,...,...
77,78,1.7
78,79,2.1
79,80,9.2
80,81,9.4


DURACIONES

In [52]:
df_duraciones=f.crear_id(datos_pelis,"durac_imbd","duraciones","id_duracion")

In [53]:
df_duraciones.head()

,id_duracion,duraciones
0,1,1h 35min
1,2,Desconocido
2,3,1h 34min
3,4,2h 33min
4,5,1h 24min


DIRECTORES

In [54]:
df_directores=f.crear_id(datos_pelis,"direc_imbd","directores","id_director")

In [55]:
df_directores.head()

,id_director,directores
0,1,Alejandro Todd
1,2,P. Vasu
2,3,Jesús Fragoso Montoya
3,4,S.A. Chandrashekhar
4,5,Desconocido


GUIONISTAS

In [56]:
df_guionistas=f.crear_id(datos_pelis,"guion_imbd","guionistas","id_guionista")

In [57]:
df_guionistas.head()

,id_guionista,guionistas
0,1,Óscar FentanesJuan Garrido
1,2,SathyarajGautamiAnandraj
2,3,Fernando Galiana
3,4,RekhaRishi KapoorJackie Shroff
4,5,Desconocido


AHORA VAMOS A CREAR LAS TABLAS INTERMEDIAS (DATOS PELIS)

In [58]:
datos_pelis.head()

,titulo,cali_imdb,direc_imbd,guion_imbd,argum_imbd,durac_imbd,id_pelicula
0,Las Travesuras de Super Chido,7.7,Alejandro Todd,Óscar FentanesJuan Garrido,Desconocido,1h 35min,tt0320656
1,Velai Kidaichiruchu,5.2,P. Vasu,SathyarajGautamiAnandraj,"Baasha, a gangster, controls a whole village w...",Desconocido,tt0318803
2,Sor Batalla,6.6,Jesús Fragoso Montoya,Fernando Galiana,As punishment for participating in a contest T...,1h 34min,tt0320503
3,Azaad Desh Ke Gulam,5.3,S.A. Chandrashekhar,RekhaRishi KapoorJackie Shroff,"Bharti, a law student from a rich family, live...",2h 33min,tt0330019
4,Suo ming fei dao,NaN,Desconocido,Desconocido,Desconocido,1h 24min,tt0348147


In [59]:
dicc_cali = df_calificaciones.set_index('calificaciones')['id_calificacion'].to_dict()
dicc_dur = df_duraciones.set_index('duraciones')['id_duracion'].to_dict()
dicc_dir = df_directores.set_index('directores')['id_director'].to_dict()
dicc_gui = df_guionistas.set_index('guionistas')['id_guionista'].to_dict()

In [60]:
datos_pelis['id_calificacion'] = datos_pelis['cali_imdb'].map(dicc_cali)
datos_pelis['id_duracion'] = datos_pelis['durac_imbd'].map(dicc_dur)
datos_pelis['id_director'] = datos_pelis['direc_imbd'].map(dicc_dir)
datos_pelis['id_guionista'] = datos_pelis['guion_imbd'].map(dicc_gui)


In [61]:
datos_pelis.drop(columns="cali_imdb", inplace=True)
datos_pelis.drop(columns="durac_imbd", inplace=True)
datos_pelis.drop(columns="direc_imbd", inplace=True)
datos_pelis.drop(columns="guion_imbd", inplace=True)

In [62]:
datos_pelis.head()

,titulo,argum_imbd,id_pelicula,id_calificacion,id_duracion,id_director,id_guionista
0,Las Travesuras de Super Chido,Desconocido,tt0320656,1,1,1,1
1,Velai Kidaichiruchu,"Baasha, a gangster, controls a whole village w...",tt0318803,2,2,2,2
2,Sor Batalla,As punishment for participating in a contest T...,tt0320503,3,3,3,3
3,Azaad Desh Ke Gulam,"Bharti, a law student from a rich family, live...",tt0330019,4,4,4,4
4,Suo ming fei dao,Desconocido,tt0348147,5,5,5,5


AHORA VAMOS A CREAR LAS TABLAS INTERMEDIAS (PELIS)

In [65]:
datos_artista.head()

,actr,año_nac,conocido_por,que_hace,premios
0,A. Gunaseelan,desconocido,'Mandela',"Casting Department, Actress, Casting Director",1 nomination
1,A. Michelle Harleston,desconocido,"'Murder by Numbers', 'The Mark of a Killer', '...",Actor,no tiene premios
2,A.C. Peterson,desconocido,"'Narc', 'Shanghai Noon', 'Shooter', 'The Last ...",Actor,no tiene premios
3,A.V.S. Subramanyam,1957,"'Roommates', 'Uncle', 'Ori Nee Prema Bangaramk...","Actor, Director, Writer",no tiene premios
4,Øyvind Øvrebø,desconocido,"'Tingen, Edderkoppen', 'Demenskoret', 'Kristia...","Actress, Additional Crew",no tiene premios


In [67]:
dicc_naci = df_nacimiento.set_index('anio_nac')['id_anio_nac'].to_dict()
dicc_prem = df_premios.set_index('premios')['id_premio'].to_dict()

In [68]:
datos_artista['id_nacimiento'] = datos_artista['año_nac'].map(dicc_naci)
datos_artista['id_premio'] = datos_artista['premios'].map(dicc_prem)



In [69]:
datos_artista.drop(columns="año_nac", inplace=True)
datos_artista.drop(columns="premios", inplace=True)


In [70]:
datos_artista.head()

,actr,conocido_por,que_hace,id_nacimiento,id_premio
0,A. Gunaseelan,'Mandela',"Casting Department, Actress, Casting Director",1,1
1,A. Michelle Harleston,"'Murder by Numbers', 'The Mark of a Killer', '...",Actor,1,2
2,A.C. Peterson,"'Narc', 'Shanghai Noon', 'Shooter', 'The Last ...",Actor,1,2
3,A.V.S. Subramanyam,"'Roommates', 'Uncle', 'Ori Nee Prema Bangaramk...","Actor, Director, Writer",2,2
4,Øyvind Øvrebø,"'Tingen, Edderkoppen', 'Demenskoret', 'Kristia...","Actress, Additional Crew",1,2


### Fase 5: Creación de una Base de Datos
Organiza toda la información recopilada en una base de datos SQL bien estructurada. Define las tablas y relaciones necesarias para almacenar los datos de manera eficiente.

---

### Fase 6: Inserción de Datos en la Base de Datos
Inserta todos los datos recopilados en la base de datos diseñada.

---

### Fase 7: Consultas SQL
Realiza consultas para responder preguntas específicas, como:
- ¿Qué géneros han recibido más premios Oscar?
- ¿Qué género tiene las mejores calificaciones en IMDB?
- ¿Qué director tiene más películas premiadas?
- ¿Cuál es la película mejor valorada en IMDB?
- ¿Qué actor/actriz ha recibido más premios?